# Lesson 1 - Brief Intro to Google ADK

This notebook shows a quick intro to Google ADK.

General steps:
1. Setup environment (use local .env and .venv)
2. Check that environment is working (including connection to Neo4j)
3. Define tool(s)
4. Define agent(s)
5. Demonstrate interaction

## Setup

### script - import libraries

- start with the usual library imports
- for this course, you'll be using Google's Agent Development Kit (ADK)
- there's also a special helper called `neo4j_for_adk` which wraps the Neo4j Python driver to make it ADK friendly

In [1]:
# Import necessary libraries
import os
from google.adk.agents import Agent
from google.adk.models.lite_llm import LiteLlm # For OpenAI support
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.genai import types # For creating message Content/Parts
from typing import Optional, Dict, Any

# Convenience libraries for working with Neo4j inside of Google ADK
from neo4j_for_adk import graphdb

import warnings
# Ignore all warnings
warnings.filterwarnings("ignore")

import logging
logging.basicConfig(level=logging.CRITICAL)

print("Libraries imported.")

Libraries imported.


### Script - prepare OpenAI

- define model to use -- openai/gpt-4o
- create a LiteLlm client to use that model
- try it out with a simple completion



In [ ]:
# --- Define Model Constants for easier use ---
MODEL_GPT_5 = "openai/gpt-5"

llm = LiteLlm(model=MODEL_GPT_5)

# Test LLM with a direct call
print(llm.llm_client.completion(model=llm.model, messages=[{"role": "user", "content": "Are you ready?"}], tools=[]))

print("\nOpenAI is ready for use.")

ModelResponse(id='chatcmpl-CNq8Th7H56L8BDIwMv7Fgl5fH4R8W', created=1759797989, model='gpt-5-2025-08-07', object='chat.completion', system_fingerprint=None, choices=[Choices(finish_reason='stop', index=0, message=Message(content='Yes—ready when you are. How can I help today?', role='assistant', tool_calls=None, function_call=None, provider_specific_fields={'refusal': None}, annotations=[]), provider_specific_fields={})], usage=Usage(completion_tokens=22, prompt_tokens=107, total_tokens=129, completion_tokens_details=CompletionTokensDetailsWrapper(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0, text_tokens=None), prompt_tokens_details=PromptTokensDetailsWrapper(audio_tokens=0, cached_tokens=0, text_tokens=None, image_tokens=None)), service_tier='default')

OpenAI is ready for use.


### script - check the connection to Neo4j

- you'll also check that Neo4j is ready
- the `graphdb` variable is a singleton exported from `neo4j_for_adk.py` 
- this is the simplest database query we can run, just returning a value

In [ ]:
# Check connection to Neo4j by sending a query

neo4j_is_ready = graphdb.send_query("RETURN 'Neo4j is Ready!' as message")

print(neo4j_is_ready)

### script - define a tool

tool definition:
- agents can use tools to perform actions
- you need to implement the action as a function
- then document the purpose of the function in a docstring

say hello tool::
- `say_hello` is a simple tool for formatting a greeting 
- it takes a `person_name` argument
- rather than directly formatting the greeting, we'll send that to Neo4j as a query parameter
- notice the `$` in the `RETURN` clause of the cypher query. that indicates a query parameter
- the second argument to `send_query` is a dictionary of parameters
- important to note that using a query parameters avoids injection attacks. 
- the `person_name` is a variable with a value. it is not a template placeholder. it never gets parsed.


In [ ]:
# Define a basic tool -- send a parameterized cypher query
def say_hello(person_name: str) -> dict:
    """Formats a welcome message to a named person. 

    Args:
        person_name (str): the name of the person saying hello

    Returns:
        dict: A dictionary containing the results of the query.
              Includes a 'status' key ('success' or 'error').
              If 'success', includes a 'query_result' key with an array of result rows.
              If 'error', includes an 'error_message' key.
    """
    return graphdb.send_query("RETURN 'Hello to you, ' + $person_name AS reply",
    {
        "person_name": person_name
    })


### script - try the tool

- since this is a simple function returning a string you can call it directly
- try it out

In [ ]:

# Example tool usage (optional test)
print(say_hello("ABK"))


### script - no injection attack

- remember that the function sends a query to Neo4j using a query parameter
- you can attempt to inject a malicious query to demonstrate that this function is safe

In [ ]:

# Example tool usage (optional test)
print(say_hello("RETURN 'injection attack avoided'"))


---

### script - define an Agent, the `hello_agent`

- an `Agent` in Google ADK orchestrates the interaction between the user, the LLM, and the available tools

- you configure it with several key parameters:
    * `name`: A unique identifier for this agent (e.g., "friendly_cypher_agent\_v1").  
    * `model`: Specifies which LLM to use. you'll use the `llm` variable we defined above.  
    * `description`: A summary of the agent's overall purpose. This is like public documentation that helps other agents decide when to delegate tasks to *this* agent.  
    * `instruction`: Detailed guidance given to the LLM on how this agent should behave, its persona, goals, and specifically *how and when* to utilize its assigned `tools`.  
    * `tools`: A list containing the actual Python tool functions the agent is allowed to use (e.g., `[say_hello]`).

**Best Practice:** Provide clear and specific `instruction` prompts. The more detailed the instructions, the better the LLM can understand its role and how to use its tools effectively. Be explicit about error handling if needed.

**Best Practice:** Choose descriptive `name` and `description` values. These are used internally by ADK and are vital for features like automatic delegation (covered later).

In [ ]:
# Define the Cypher Agent

hello_agent = Agent(
    name="hello_agent_v1",
    model=llm, # defined earlier in a variable
    description="Has friendly chats with a user.",
    instruction="""You are a helpful assistant, chatting with a user. 
                Be polite and friendly, introducing yourself and asking who the user is. 

                If the user provides their name, use the 'say_hello' tool to get a custom greeting.
                If the tool returns an error, inform the user politely. 
                If the tool is successful, present the reply.
                """,
    tools=[say_hello], # Pass the function directly
)

print(f"Agent '{hello_agent.name}' created.")

---

### Script - Set up AgentCaller

To run an agent, you'll need some additional components which we'll wrap in helpers.

An agent needs an execution environment and memory. We'll define those in the next step.

This `AgentCaller` class is a helper that makes it easier to make repeated calls to the
agent by assuming we have a single user talking to the agent in a single session.

Internally, here's what's happening in the `call` function:

1. Takes a user query string.  
2. Packages it into the ADK `Content` format.  
3. Calls `runner.run_async`, providing the user/session context and the new message.  
4. Iterates through the **Events** yielded by the runner. Events represent steps in the agent's execution (e.g., tool call requested, tool result received, intermediate LLM thought, final response).  
5. Identifies and prints the **final response** event using `event.is_final_response()`.

**Why `async`?** Interactions with LLMs and potentially tools (like external APIs) are I/O-bound operations. Using `asyncio` allows the program to handle these operations efficiently without blocking execution.

In [ ]:
class AgentCaller:
    """A simple wrapper class for interacting with an ADK agent."""
    
    def __init__(self, agent: Agent, runner: Runner, user_id: str, session_id: str):
        """Initialize the AgentCaller with required components."""
        self.agent = agent
        self.runner = runner
        self.user_id = user_id
        self.session_id = session_id


    def get_session(self):
        return self.runner.session_service.get_session(app_name=self.runner.app_name, user_id=self.user_id, session_id=self.session_id)

    
    async def call(self, user_message: str, verbose: bool = False):
        """Call the agent with a query and return the response."""
        print(f"\n>>> User Message: {user_message}")

        # Prepare the user's message in ADK format
        content = types.Content(role='user', parts=[types.Part(text=user_message)])

        final_response_text = "Agent did not produce a final response." # Default will be replaced if the agent produces a final response.

        # Key Concept: run_async executes the agent logic and yields Events.
        # We iterate through events to find the final answer.
        async for event in self.runner.run_async(user_id=self.user_id, session_id=self.session_id, new_message=content):
            # You can uncomment the line below to see *all* events during execution
            if verbose:
                print(f"  [Event] Author: {event.author}, Type: {type(event).__name__}, Final: {event.is_final_response()}, Content: {event.content}")

            # Key Concept: is_final_response() marks the concluding message for the turn.
            if event.is_final_response():
                if event.content and event.content.parts:
                    # Assuming text response in the first part
                    final_response_text = event.content.parts[0].text
                elif event.actions and event.actions.escalate: # Handle potential errors/escalations
                    final_response_text = f"Agent escalated: {event.error_message or 'No specific message.'}"
                break # Stop processing events once the final response is found

        print(f"<<< Agent Response: {final_response_text}")
        return final_response_text


### script - make an instance of the AgentCaller

Rather than a class constructor, you'll use a factory method which needs to make some async calls to initialize the components.

The factory method takes some parameters:

* `Agent`: the agent that we defined earlier
* `initial_state`: optional initialization of the agent's "memory"

Inside, the method will create memory for the agent and a runner.

* `SessionService`: Responsible for managing conversation history and state for different users and sessions. The `InMemorySessionService` is a simple implementation that stores everything in memory, suitable for testing and simple applications. It keeps track of the messages exchanged.  
* `Runner`: The engine that orchestrates the interaction flow. It takes user input, routes it to the appropriate agent, manages calls to the LLM and tools based on the agent's logic, handles session updates via the `SessionService`, and yields events representing the progress of the interaction.


In [ ]:

async def make_agent_caller(agent: Agent, initial_state: Optional[Dict[str, Any]] = {}) -> AgentCaller:
    """Create and return an AgentCaller instance for the given agent."""
    app_name = agent.name + "_app"
    user_id = agent.name + "_user"
    session_id = agent.name + "_session_01"
    
    # Initialize a session service and a session
    session_service = InMemorySessionService()
    await session_service.create_session(
        app_name=app_name,
        user_id=user_id,
        session_id=session_id,
        state=initial_state
    )
    
    runner = Runner(
        agent=agent,
        app_name=app_name,
        session_service=session_service
    )
    
    return AgentCaller(agent, runner, user_id, session_id)




---

### Script - Run the Conversation

Now you can define an async function to run the conversation.

Watch the output:

* See the user messages.  
* Notice the agent responses.
* Observe the agent's final responses, including how it handles the case when a user name is not provided.

In [ ]:
hello_agent_caller = await make_agent_caller(hello_agent)

# We need an async function to await our interaction helper
async def run_conversation():
    await hello_agent_caller.call("Hello I'm ABK")

    await hello_agent_caller.call("I am excited")

# Execute the conversation using await
await run_conversation()

---

Congratulations\! You've successfully built and interacted with your first ADK agent. It understands the user's request, uses a tool to find information, and responds appropriately based on the tool's result.

Next, we'll build a simple multi-agent system.

## A Simple Multi-Agent Team \- Delegation for Greetings & Farewells

Using multiple agents is a common pattern in real-world applications. It allows for better modularity, specialization, and scalability. 

Here's how it works:

1. Creating multiple, **specialized agents**, each designed for a specific capability 
    - you'll create one agent in charge of handling greetings
    - then another used to handle farewells
2. Designating a **root agent** (or orchestrator or coordinator) that receives the initial user request.  
3. Enabling the root agent to **delegate** the request to the most appropriate specialized sub-agent based on the user's intent.

**Why build an Agent Team?**

* **Modularity:** Easier to develop, test, and maintain individual agents.  
* **Specialization:** Each agent can be fine-tuned (instructions, model choice) for its specific task.  
* **Scalability:** Simpler to add new capabilities by adding new agents.  
* **Efficiency:** Allows using potentially simpler/cheaper models for simpler tasks (like greetings).

**To illustrate this, you will:**

1. Define another simple tool that will handle farewells (`say_goodbye`).  
2. Create two new specialized sub-agents: `greeting_agent` and `farewell_agent`.  
3. Create a new top-level agent (`cypher_agent_team`) to act as the **root agent**.  
4. Configure the root agent with its sub-agents, enabling **automatic delegation**.  
5. Test the delegation flow by sending different types of requests to the root agent.

---

**1\. Define a new tool**

First, let's create the simple Python functions that will serve as tools for our new specialist agents. Remember, clear docstrings are vital for the agents that will use them.

In [ ]:
# Define the new goodbye tool

def say_goodbye() -> dict:
    """Provides a simple farewell message to conclude the conversation."""
    return graphdb.send_query("RETURN 'Goodbye from Cypher!' as farewell")


In [ ]:

# Try it out
print(say_goodbye())

---

**2\. Define the Sub-Agents (Greeting & Farewell)**

Now, create the `Agent` instances for our specialists. Notice their highly focused `instruction` and, critically, their clear `description`. The `description` is the primary information the *root agent* uses to decide *when* to delegate to these sub-agents.

**Best Practice:** Sub-agent `description` fields should accurately and concisely summarize their specific capability. This is crucial for effective automatic delegation.

**Best Practice:** Sub-agent `instruction` fields should be tailored to their limited scope, telling them exactly what to do and *what not* to do (e.g., "Your *only* task is...").

In [ ]:
# --- Greeting Agent ---

# Like the `hello_agent` from before, but now expected to be used as a sub-agent.
# Should do _nothing_ other than generate greetings using the `say_hello` tool.
greeting_subagent = Agent(
    model=llm,
    name="greeting_subagent_v1",
    instruction="You are the Greeting Agent. Your ONLY task is to provide a friendly greeting to the user. "
                "Use the 'say_hello' tool to generate the greeting. "
                "If the user provides their name, make sure to pass it to the tool. "
                "Do not engage in any other conversation or tasks.",
    description="Handles simple greetings and hellos using the 'say_hello' tool.", # Crucial for delegation
    tools=[say_hello],
)
print(f"✅ Agent '{greeting_subagent.name}' created.")


In [ ]:
# --- Farewell Agent ---
# A new sub-agent to say goodbye. 
# Like the `greeting_subagent` this one should do nothing other than say goodbye
# using the `say_goodbye` tool.
# 
# Note: these comments are a lot like the `instruction` :) 
farewell_subagent = Agent(
    # Can use the same or a different model
    model=llm, # Sticking with GPT for this example
    name="farewell_subagent_v1",
    instruction="You are the Farewell Agent. Your ONLY task is to provide a polite goodbye message. "
                "Use the 'say_goodbye' tool when the user indicates they are leaving or ending the conversation "
                "(e.g., using words like 'bye', 'goodbye', 'thanks bye', 'see you'). "
                "Do not perform any other actions.",
    description="Handles simple farewells and goodbyes using the 'say_goodbye' tool.", # Crucial for delegation
    tools=[say_goodbye],
)
print(f"✅ Agent '{farewell_subagent.name}' created.")


---

### Define the Root Agent (`cypher_agent_team`) with Sub-Agents

Our `cypher_agent_team` is now a root agent with sub-agents. The key changes are:

* Adding the `sub_agents` parameter: We pass a list containing the `greeting_agent` and `farewell_agent` instances we just created.  
* Updating the `instruction`: We explicitly tell the root agent *about* its sub-agents and *when* it should delegate tasks to them.

**Key Concept: Automatic Delegation (Auto Flow)** By providing the `sub_agents` list, ADK enables automatic delegation. When the root agent receives a user query, its LLM considers not only its own instructions and tools but also the `description` of each sub-agent. If the LLM determines that a query aligns better with a sub-agent's described capability (e.g., "Handles simple greetings"), it will automatically generate a special internal action to *transfer control* to that sub-agent for that turn. The sub-agent then processes the query using its own model, instructions, and tools.

**Best Practice:** Ensure the root agent's instructions clearly guide its delegation decisions. Mention the sub-agents by name and describe the conditions under which delegation should occur.

In [ ]:

# --- Friendly Agent team ---
# A "root" or "top-level" or "coordinator" agent that will
# get interactions going and "delegate" to sub-agents for
# specialized tasks like saying hello or goodbye.

friendly_agent_team = Agent(
    name="friendly_agent_team_v1", # versions in the name are convenient for sanity checking
    model=llm,
    description="The main coordinator agent. Delegates greetings/farewells to specialists.",
    instruction="""You are the main Agent coordinating a team. Your primary responsibility is to be friendly.

                You have specialized sub-agents: 
                1. 'greeting_agent': Handles simple greetings like 'Hi', 'Hello'. Delegate to it for these. 
                2. 'farewell_agent': Handles simple farewells like 'Bye', 'See you'. Delegate to it for these. 

                Analyze the user's query. If it's a greeting, delegate to 'greeting_agent'. If it's a farewell, delegate to 'farewell_agent'. 
                
                For anything else, respond appropriately or state you cannot handle it.
                """,
    tools=[], # No tools for the root agent
    
    # *Key change*: Link the sub-agents here!
    sub_agents=[greeting_subagent, farewell_subagent]
)


print(f"✅ Root Agent '{friendly_agent_team.name}' created with sub-agents: {[sa.name for sa in friendly_agent_team.sub_agents]}")


---

### script - Interact with the Agent Team

Now that we've defined our root agent (`cypher_agent_team` - *Note: Ensure this variable name matches the one defined in the previous code block, likely `# @title Define the Root Agent with Sub-Agents`, which might have named it `root_agent`*) with its specialized sub-agents, let's test the delegation mechanism.

The following code block will:

1.  Define an `async` function `run_team_conversation`.
2.  Inside this function, create a *new, dedicated* `InMemorySessionService` and a specific session (`session_001_agent_team`) just for this test run. This isolates the conversation history for testing the team dynamics.
3.  Create a `Runner` (`runner_agent_team`) configured to use our `cypher_agent_team` (the root agent) and the dedicated session service.
4.  Use our updated `call_agent_async` function to send different types of queries (greeting, farewell) to the `runner_agent_team`. We explicitly pass the runner, user ID, and session ID for this specific test.
5.  Immediately execute the `run_team_conversation` function.

We expect the following flow:

1.  The "Hello there!" query goes to `runner_agent_team`.
2.  The root agent (`cypher_agent_team`) receives it and, based on its instructions and the `greeting_agent`'s description, delegates the task.
3.  `greeting_agent` handles the query, calls its `say_hello` tool, and generates the response.
4.  The "What is Neo4j?" query is *not* delegated and is handled directly by the root agent using its public knowledge.
5.  The "Thanks, bye!" query is delegated to the `farewell_agent`, which uses its `say_goodbye` tool.



In [ ]:
friendly_team_caller = await make_agent_caller(friendly_agent_team)

async def run_team_conversation():
    # run the conversation with verbose output to see the action
    await friendly_team_caller.call("Hello I'm ABK", True)

    await friendly_team_caller.call("Thanks, bye!", True)

# Execute the conversation using await
await run_team_conversation()


---

Look closely at the output logs, especially the `--- Tool: ... called ---` messages. You should observe:

*   For "Hello there!", the `say_hello` tool was called (indicating `greeting_agent` handled it).
*   For "What is Neo4j?",  the agent handled it directly without a function call.
*   For "Thanks, bye!", the `say_goodbye` tool was called (indicating `farewell_agent` handled it).

This confirms successful **automatic delegation**! The root agent, guided by its instructions and the `description`s of its `sub_agents`, correctly routed user requests to the appropriate specialist agent within the team.

You've now structured your application with multiple collaborating agents. This modular design is fundamental for building more complex and capable agent systems. In the next step, we'll give our agents the ability to remember information across turns using session state.

## Step 4: Adding Memory and Personalization with Session State

So far, our agent team can handle different tasks through delegation, but each interaction starts fresh – the agents have no memory of past conversations or user preferences within a session. To create more sophisticated and context-aware experiences, agents need **memory**. ADK provides this through **Session State**.

**What is Session State?**

* It's a Python dictionary (`session.state`) tied to a specific user session (identified by `APP_NAME`, `USER_ID`, `SESSION_ID`).  
* It persists information *across multiple conversational turns* within that session.  
* Agents and Tools can read from and write to this state, allowing them to remember details, adapt behavior, and personalize responses.

**How Agents Interact with State:**

1. **`ToolContext` (Primary Method):** Tools can accept a `ToolContext` object (automatically provided by ADK if declared as the last argument). This object gives direct access to the session state via `tool_context.state`, allowing tools to read preferences or save results *during* execution.  
2. **`output_key` (Auto-Save Agent Response):** An `Agent` can be configured with an `output_key="your_key"`. ADK will then automatically save the agent's final textual response for a turn into `session.state["your_key"]`.

**In this step, we will enhance our Cypher team by:**

1. Using a **new** `InMemorySessionService` to demonstrate state in isolation.  
2. Initializing session state with a default user name.  
3. Creating a state-aware version of the say_hello tool (`say_hello_stateful`) which responds with a personalized greeting and sets the user name in the session state.  
4. Creating a state-aware version of the say_goodbye tool (`say_goodbye_stateful`) that reads this preference via `ToolContext` and adjusts a personalized goodbye message.  
5. Running a conversation to observe how the initial state affects the tool, how manual state changes alter subsequent behavior.

---

### script - Create State-Aware hello/goodbye Tools (`say_hello_stateful` and `say_goodbye_stateful`)

You will create a new version of the hello/goodbye tools. The key feature is accepting `tool_context: ToolContext` 
which allows them to access `tool_context.state`. 

They will write to or read from the `user_name` state variable.


* **Key Concept: `ToolContext`** This object is the bridge allowing your tool logic to interact with the session's context, including reading and writing state variables. ADK injects it automatically if defined as the last parameter of your tool function.


* **Best Practice:** When reading from state, use `dictionary.get('key', default_value)` to handle cases where the key might not exist yet, ensuring your tool doesn't crash.

In [ ]:
from google.adk.tools.tool_context import ToolContext

def say_hello_stateful(user_name:str, tool_context:ToolContext):
    """Says hello to the user, recording their name into state.
    
    Args:
        user_name (str): The name of the user.
    """
    # store the user name in state/memory
    tool_context.state["user_name"] = user_name
    
    return graphdb.send_query(
        f"RETURN 'Hello to you, ' + $user_name + '.' AS reply",
    {
        "user_name": user_name
    })
    
# can't call this tool directly, as it uses `ToolContext` to access state

In [ ]:
def say_goodbye_stateful(tool_context: ToolContext) -> dict:
    """Says goodbye to the user, reading their name from state."""
    # get ths uername from state/memory _or_ use a default value
    user_name = tool_context.state.get("user_name", "stranger")
    print("\ntool_context.state['user_name']:", user_name)
    return graphdb.send_query("RETURN 'Goodbye, ' + $user_name + ', nice to chat with you!' AS reply",
    {
        "user_name": user_name
    })


print("✅ State-aware 'say_hello_stateful' and 'say_goodbye_stateful' tools defined.")


---

**3\. Redefine Sub-Agents and Update Root Agent**

To ensure this step is self-contained and builds correctly, we first redefine the `greeting_agent` and `farewell_agent` exactly as they were in Step 3\. Then, we define our new root agent (`cypher_agent_v4_stateful`):

* It uses the new `say_hello_stateful` and `say_goodbye_stateful` tools.  
* It includes the greeting and farewell sub-agents for delegation.  


In [ ]:
# define a stateful greeting agent. the only difference is that this agent will use the stateful say_hello_stateful tool
greeting_agent_stateful = Agent(
    model=llm,
    name="greeting_agent_stateful_v1",
    instruction="You are the Greeting Agent. Your ONLY task is to provide a friendly greeting using the 'say_hello' tool. Do nothing else.",
    description="Handles simple greetings and hellos using the 'say_hello_stateful' tool.",
    tools=[say_hello_stateful],
)
print(f"✅ Agent '{greeting_agent_stateful.name}' redefined.")


In [ ]:

farewell_agent_stateful = Agent(
    model=llm,
    name="farewell_agent_stateful_v1",
    instruction="You are the Farewell Agent. Your ONLY task is to provide a polite goodbye message using the 'say_goodbye_stateful' tool. Do not perform any other actions.",
    description="Handles simple farewells and goodbyes using the 'say_goodbye_stateful' tool.",
    tools=[say_goodbye_stateful],
)
print(f"✅ Agent '{farewell_agent_stateful.name}' redefined.")

In [ ]:

friendly_team_stateful = Agent(
    name="friendly_team_stateful", # New version name
    model=llm,
    description="The main coordinator agent. Delegates greetings/farewells to specialists.",
    instruction="""You are the main Agent coordinating a team. Your primary responsibility is to be friendly.

                You have specialized sub-agents: 
                1. 'greeting_agent_stateful': Handles simple greetings like 'Hi', 'Hello'. Delegate to it for these. 
                2. 'farewell_agent_stateful': Handles simple farewells like 'Bye', 'See you'. Delegate to it for these. 

                Analyze the user's query. If it's a greeting, delegate to 'greeting_agent_stateful'. If it's a farewell, delegate to 'farewell_agent_stateful'. 
                
                For anything else, respond appropriately or state you cannot handle it.
                """,
        tools=[], # Still no tools for root
        sub_agents=[greeting_agent_stateful, farewell_agent_stateful], # Include sub-agents
    )

print(f"✅ Root Agent '{friendly_team_stateful.name}' created using agents with stateful tools.")


---

### script - Interact and Test State Flow

Now, you can initialize a new `AgentCaller`. This time, you'll provide an initial state.

Then you can execute a conversation designed to test the state interactions using the `cypher_team_stateful` root agent.

These are the steps you'll take:

1. create an AgentCaller instance for the `cypher_team_stateful` agent with an initial state where the `user_name` is set to "unknown"
2. display the initial state of the session that the AgentCaller will use
3. set up a conversation where the user will introduce themselves then say goodbye
4. show the final state of the session, which should now have an updated `user_name`

In [ ]:
friendly_team_stateful_caller = await make_agent_caller(friendly_team_stateful)

# get the session to see the state
session = await friendly_team_stateful_caller.get_session()

print(f"Initial State: {session.state}")

Now, you can define a conversation, run it, then examine the final session state.

In [ ]:
async def run_stateful_conversation():
    await friendly_team_stateful_caller.call("Hello, I'm ABK!", True)

    await friendly_team_stateful_caller.call("Thanks, bye!", True)

# Execute the conversation using await in an async context (like Colab/Jupyter)
await run_stateful_conversation()

session = await friendly_team_stateful_caller.get_session()

print(f"\nFinal State: {session.state}")

---

By reviewing the conversation flow and the final session state printout, you can confirm:

*   **State Update:** The `say_hello_stateful` tool successfully updated `user_name` in state.
*   **State Read:** The `say_goodbye_stateful` tool correctly read `user_name` from state.
*   **Delegation:** The delegation to the `greeting_agent` for "Hi!" functioned correctly even after state modifications.
*   **Final State:** The final check confirms the user_name persisted as "ABK".

You've now successfully integrated session state to personalize agent behavior using `ToolContext`, manually manipulated state for testing `InMemorySessionService`, and observed the behavior. 

This foundational understanding of state management is key as we proceed to implementing a stateful workflow..

---

---
## For Fun, An Interactive Conversation

Now, you can make this interactive to see how it behaves! 

Run the cell below. It will prompt you to enter your queries directly.

Type `exit` to quit the conversation.

In [ ]:
async def run_interactive_conversation():
    while True:
        user_query = input("Ask me something (or type 'exit' to quit): ")
        if user_query.lower() == 'exit':
            break
        response = await friendly_team_stateful_caller.call(user_query)
        print(f"Response: {response}")

# Execute the interactive conversation
await run_interactive_conversation()